In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, Input, MaxPooling2D
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report, confusion_matrix

# Create a new generator
imagegen = ImageDataGenerator(rotation_range=10, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True, brightness_range=[0.2, 1.2])

# Load train data
train = imagegen.flow_from_directory("/content/drive/MyDrive/mega project/Dataset/train", class_mode="categorical", shuffle=False, batch_size=128)

# Load val data
val = imagegen.flow_from_directory("/content/drive/MyDrive/mega project/Dataset/val", class_mode="categorical", shuffle=False, batch_size=128)

test_path = '/content/drive/MyDrive/mega project/Dataset/test'

testing = []
for i in os.listdir(test_path):
  if i[:3] == 'nor':
    testing.append('Normal')
  else:
    testing.append('Pothole')

input_img = Input(shape=(256, 256, 3))

tower_1 = Conv2D(25, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(25, (3, 3), padding='same', activation='relu')(tower_1)
tower_2 = Conv2D(25, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(25, (5, 5), padding='same', activation='relu')(tower_2)
tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(25, (1, 1), padding='same', activation='relu')(tower_3)
output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=3)
output = Flatten()(output)
out = Dense(2, activation='softmax')(output)
model1 = Model(inputs=input_img, outputs=out)

# Finding the best model
mc1 = ModelCheckpoint('best_model_inception.h5', monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)

# Compile model
model1.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# Fit on data for 50 epochs
history1 = model1.fit(train, epochs=20, validation_data=val, callbacks=[mc1])

# Loading the best model
model1 = load_model('best_model_inception.h5')

# Model summary
model1.summary()

(eval_loss1, eval_accuracy1) = model1.evaluate(val, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy1 * 100))
print("[INFO] Loss: {}".format(eval_loss1))

# Draw the model architecture
plot_model(model1, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

# Display the model architecture
from IPython.display import Image
Image('model_architecture.png')


# Predict classes for validation data
val_pred = model1.predict(val)
val_pred_classes = np.argmax(val_pred, axis=1)

# True classes for validation data
true_classes = val.classes

# Class labels
class_labels = list(val.class_indices.keys())

# Classification report
print("Classification Report:")
print(classification_report(true_classes, val_pred_classes, target_names=class_labels))

# Confusion matrix
print("Confusion Matrix:")
conf_matrix = confusion_matrix(true_classes, val_pred_classes)
print(conf_matrix)

# Accuracy
accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
print("Accuracy:", accuracy)

# Precision
precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
print("Precision:", precision)

# Recall
recall = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
print("Recall:", recall)

# F1-score
f1_score = 2 * (precision * recall) / (precision + recall)
print("F1-Score:", f1_score)

# Plotting training and validation accuracy
plt.plot(history1.history['accuracy'], label='Training Accuracy')
plt.plot(history1.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(history1.history['loss'], label='Training Loss')
plt.plot(history1.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Exception ignored in: <function _xla_gc_callback at 0x7ebe393f12d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Found 1157 images belonging to 2 classes.
Found 108 images belonging to 2 classes.


KeyboardInterrupt: 